# Diversity analysis

In [83]:
# Diversity analysis

## I Data preparation

In [84]:
# I Data preparation

### Loading packages

In [69]:
# Loading Packages
library("diversitree")
library("stringr")
library("qpcR")
library("ggtree")
library("ggplot2")
library("ggpmisc")

Le chargement a nécessité le package : MASS

Le chargement a nécessité le package : minpack.lm

Le chargement a nécessité le package : rgl

This build of rgl does not include OpenGL functions.  Use
 rglwidget() to display results, e.g. via options(rgl.printRglwidget = TRUE).

Le chargement a nécessité le package : robustbase

Le chargement a nécessité le package : Matrix


Attachement du package : ‘Matrix’


L'objet suivant est masqué depuis ‘package:ggtree’:

    expand




### Loading data

In [4]:
# Loading Packages
df<-read.csv("Path/to/your/file.tsv", sep="\t")
nex<-read.nexus("Path/to/your/tree.tree")

### Aicc function

In [5]:
# Aicc function
aicc<-function(ll,k){round((2*(-round(ll,4))+2*k+(2*k*(k+1))/(Ntip(nex)-k-1)),3)}

### Optional step (cleaning data)

In [9]:
# Optional step (cleaning data)
rownames(df)<-str_replace(rownames(df), " ", "_")
states<-as.numeric(df$clust.num)
names(states)<-rownames(df)
states2<-states[!names(states) %in% setdiff(names(states), nex$tip.label)]
nex3<-drop.tip(nex, setdiff(nex$tip.label, (names(states))))

### Computing sampling fractions

In [16]:
# Computing sampling fractions
samp_frac<-c(
length(states2[states2==1])/length(df$clust.num[df$clust.num==1]),
length(states2[states2==2])/length(df$clust.num[df$clust.num==2]),
length(states2[states2==3])/length(df$clust.num[df$clust.num==3]),
length(states2[states2==4])/length(df$clust.num[df$clust.num==4]),
length(states2[states2==5])/length(df$clust.num[df$clust.num==5]))

## II Likelihood analysis

### initialization

In [17]:
# initialization
lik<-make.musse(nex3, states, 5, sampling.f = samp_frac, strict=TRUE)
p <- starting.point.musse(nex3, 5)

### Complete model

In [19]:
# Complete model
lik.all.free<-constrain(lik)
fit.all.free<-find.mle(lik.all.free, p[argnames(lik.all.free)])

In [20]:
p<-fit.all.free$par

### Null model

In [21]:
# Null model
lik.base <- constrain(lik,
lambda5 ~ lambda1, lambda4 ~ lambda1, lambda3 ~ lambda1, lambda2 ~ lambda1,
mu5 ~ mu1, mu4 ~ mu1, mu3 ~ mu1, mu2 ~ mu1,
q15 ~ q12, q14 ~ q12, q13 ~ q12,
q25 ~ q12, q24 ~ q12, q23 ~ q12, q21 ~ q12,
q35 ~ q12, q34 ~ q12, q32 ~ q12, q31 ~ q12,
q45 ~ q12, q43 ~ q12, q42 ~ q12, q41 ~ q12,
q54 ~ q12, q53 ~ q12, q52 ~ q12, q51 ~ q12)

fit.base <- find.mle(lik.base, p[argnames(lik.base)])

### Free speciation model

In [22]:
# Free speciation model
lik.sp <- constrain(lik,
mu5 ~ mu1, mu4 ~ mu1, mu3 ~ mu1, mu2 ~ mu1,
q15 ~ q12, q14 ~ q12, q13 ~ q12,
q25 ~ q12, q24 ~ q12, q23 ~ q12, q21 ~ q12,
q35 ~ q12, q34 ~ q12, q32 ~ q12, q31 ~ q12,
q45 ~ q12, q43 ~ q12, q42 ~ q12, q41 ~ q12,
q54 ~ q12, q53 ~ q12, q52 ~ q12, q51 ~ q12)
fit.sp <- find.mle(lik.sp, p[argnames(lik.sp)])

### Free extinction model

In [23]:
# Free extinction model
lik.ext <- constrain(lik,
lambda5 ~ lambda1, lambda4 ~ lambda1, lambda3 ~ lambda1, lambda2 ~ lambda1,
q15 ~ q12, q14 ~ q12, q13 ~ q12,
q25 ~ q12, q24 ~ q12, q23 ~ q12, q21 ~ q12,
q35 ~ q12, q34 ~ q12, q32 ~ q12, q31 ~ q12,
q45 ~ q12, q43 ~ q12, q42 ~ q12, q41 ~ q12,
q54 ~ q12, q53 ~ q12, q52 ~ q12, q51 ~ q12)           
fit.ext <- find.mle(lik.ext, p[argnames(lik.ext)])

### Free transition model

In [24]:
# Free transition model
lik.qt<-constrain(lik,                  
lambda5 ~ lambda1, lambda4 ~ lambda1, lambda3 ~ lambda1, lambda2 ~ lambda1,
mu5 ~ mu1, mu4 ~ mu1, mu3 ~ mu1, mu2 ~ mu1)
fit.qt <- find.mle(lik.qt, p[argnames(lik.qt)])

### Free speciation-extinction model

In [25]:
# Free speciation-extinction model
lik.sp_ext<-constrain(lik,
q15 ~ q12, q14 ~ q12, q13 ~ q12,
q25 ~ q12, q24 ~ q12, q23 ~ q12, q21 ~ q12,
q35 ~ q12, q34 ~ q12, q32 ~ q12, q31 ~ q12,
q45 ~ q12, q43 ~ q12, q42 ~ q12, q41 ~ q12,
q54 ~ q12, q53 ~ q12, q52 ~ q12, q51 ~ q12)
fit.sp_ext<- find.mle(lik.sp_ext, p[argnames(lik.sp_ext)])

### Free speciation-transition model

In [26]:
# Free speciation-transition model
lik.sp_qt<-constrain(lik, 
mu5 ~ mu1, mu4 ~ mu1, mu3 ~ mu1, mu2 ~ mu1)
fit.sp_qt<- find.mle(lik.sp_qt, p[argnames(lik.sp_qt)])

### Free transition-extinction model

In [27]:
# Free transition-extinction model
lik.qt_ext<-constrain(lik,
lambda5 ~ lambda1, lambda4 ~ lambda1, lambda3 ~ lambda1, lambda2 ~ lambda1)
fit.qt_ext<- find.mle(lik.qt_ext, p[argnames(lik.qt_ext)])

### Comparaison table

In [81]:
# Comparaison table
results<-matrix(NA,8,35) 


colnames(results)<-c("Nbparam","logL","AICc","DeltaAIC", "WAIC",argnames(lik))

rownames(results)<-c("base","sp","ext","qt","sp_qt","sp_ext","qt_ext","all.free")

for(j in 1:length(rownames(results)))
{
eval(parse(text=paste("results[j,names(fit.",rownames(results)[j],"$par)]<-fit.",rownames(results)[j],"$par",sep="")))
eval(parse(text=paste("results[j,'Nbparam']<-length(fit.",rownames(results)[j],"$par)",sep="")))
eval(parse(text=paste("results[j,'logL']<-fit.",rownames(results)[j],"$lnLik",sep="")))
eval(parse(text=paste("results[j,'AICc']<-aicc(fit.",rownames(results)[j],"$lnLik,length(fit.",rownames(results)[j],"$par) )",sep="")))
}

results[,4] <- round(akaike.weights(results[,3])$deltaAIC, 3)
results[,5] <- round(akaike.weights(results[,3])$weights, 3)
print(results)

         Nbparam      logL     AICc DeltaAIC  WAIC    lambda1     lambda2
base           3 -2144.127 4294.318   92.217 0.000 0.03865684          NA
sp             7 -2128.172 4270.648   68.547 0.000 0.02935205 0.017333184
ext            7 -2135.364 4285.032   82.931 0.000 0.03716657          NA
qt            22 -2088.656 4224.179   22.078 0.000 0.03872056          NA
sp_qt         26 -2073.039 4202.101    0.000 0.627 0.03460340 0.016135989
sp_ext        11 -2126.735 4276.195   74.094 0.000 0.02831294 0.013285327
qt_ext        26 -2078.654 4213.330   11.229 0.002 0.03633808          NA
all.free      30 -2068.878 4203.148    1.047 0.371 0.03605497 0.009828589
            lambda3    lambda4    lambda5        mu1          mu2         mu3
base             NA         NA         NA 0.02059864           NA          NA
sp       0.04571634 0.02698842 0.03927568 0.01311117           NA          NA
ext              NA         NA         NA 0.02150052 3.031865e-02 0.005246088
qt               NA   

### Saving data

In [82]:
# Saving data
write.csv(results, "Path/to/Result_tab.csv", sep ="\t")

Warning message in write.csv(results, "Result_tab.csv", sep = "\t"):
“une tentative de modification de 'sep' a échoué”


## III Bayesian analysis

In [ ]:
# III Bayesian analysis

### Setting priors

In [111]:
# Setting priors
p<-fit.sp_qt$par
prior <- make.prior.exponential(1/(2*(p[3]-p[8])))

### Running the analysis for 1000 generation and screening every 10 generation

In [112]:
# Running the analysis for 1000 generation and screening every 10 generation
set.seed(1)
tmp <- mcmc(lik.sp_qt, p, nsteps=100, prior=prior, w=1, print.every=10)

w <- diff(sapply(tmp[2:(length(tmp)-1)], quantile, c(0.025, 0.975)))

mcmc_MuSSE <- mcmc(lik.sp_qt, p, nsteps=1000, prior=prior, w=w)
save(mcmc_MuSSE, file="Path/to/Your_MCMC_DATA")

10: {0.0204, 0.0077, 0.0448, 0.0228, 0.0450, 0.0096, 0.0001, 0.0025, 0.0018, 0.0005, 0.0014, 0.0005, 0.0000, 0.0027, 0.0058, 0.0002, 0.0002, 0.0000, 0.0017, 0.0007, 0.0011, 0.0032, 0.0111, 0.0081, 0.0007, 0.0007} -> -2022.62219
20: {0.0321, 0.0098, 0.0483, 0.0270, 0.0520, 0.0190, 0.0000, 0.0033, 0.0009, 0.0013, 0.0006, 0.0001, 0.0005, 0.0024, 0.0070, 0.0003, 0.0003, 0.0000, 0.0014, 0.0012, 0.0003, 0.0005, 0.0074, 0.0060, 0.0004, 0.0001} -> -2015.31117
30: {0.0335, 0.0161, 0.0567, 0.0303, 0.0481, 0.0213, 0.0001, 0.0036, 0.0005, 0.0013, 0.0006, 0.0002, 0.0017, 0.0041, 0.0070, 0.0001, 0.0003, 0.0002, 0.0027, 0.0046, 0.0005, 0.0002, 0.0107, 0.0047, 0.0004, 0.0008} -> -2022.04168
40: {0.0252, 0.0187, 0.0506, 0.0300, 0.0387, 0.0130, 0.0002, 0.0042, 0.0012, 0.0003, 0.0016, 0.0000, 0.0002, 0.0019, 0.0066, 0.0001, 0.0000, 0.0006, 0.0035, 0.0005, 0.0005, 0.0004, 0.0067, 0.0047, 0.0003, 0.0008} -> -2016.47981
50: {0.0300, 0.0100, 0.0481, 0.0218, 0.0484, 0.0155, 0.0005, 0.0022, 0.0019, 0.0007, 0.0

,i,lambda1,lambda2,lambda3,lambda4,lambda5,mu1,q12,q13,q14,⋯,q35,q41,q42,q43,q45,q51,q52,q53,q54,p
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0.03620787,0.017483676,0.04681462,0.02614782,0.03691689,0.016400518,2.403810e-04,0.004770491,0.0005820168,⋯,0.0001135089,2.627563e-03,0.0017691843,6.200894e-04,0.0006968468,0.001953943,0.006931485,1.697486e-04,0.0019090643,-2020.784
2,2,0.03543199,0.001719977,0.04622261,0.03111013,0.03424680,0.016217841,1.089712e-04,0.004613937,0.0006251376,⋯,0.0001457422,2.497395e-03,0.0005783397,1.006540e-04,0.0002509319,0.001460223,0.006982346,2.074584e-06,0.0019103783,-2022.054
3,3,0.03342839,0.018333689,0.04905955,0.02318538,0.03448981,0.009947735,2.503462e-04,0.004784460,0.0007035154,⋯,0.0003316982,3.952196e-03,0.0020999321,1.034077e-03,0.0003754451,0.006508062,0.005123824,4.928861e-04,0.0015689534,-2030.262
4,4,0.02795757,0.016137892,0.05079364,0.02223164,0.03487157,0.010695484,1.569721e-04,0.004438661,0.0006983764,⋯,0.0003425077,6.794759e-04,0.0013410336,3.720642e-06,0.0004800225,0.009276937,0.006818117,1.100107e-03,0.0011700381,-2023.809
5,5,0.02932627,0.012963263,0.04332283,0.02599110,0.04113876,0.013402825,1.549874e-04,0.002636696,0.0010828104,⋯,0.0002694401,3.864286e-04,0.0001568424,8.385534e-05,0.0025496198,0.008926587,0.003756915,7.144808e-04,0.0002327657,-2016.781
6,6,0.02511681,0.013599387,0.04212505,0.03220082,0.04202040,0.013251545,3.086602e-05,0.004923565,0.0013138296,⋯,0.0001250283,9.052784e-05,0.0011326754,1.712423e-03,0.0025626647,0.008045014,0.003969026,1.143213e-03,0.0015751875,-2021.695


1: {0.0367, 0.0171, 0.0414, 0.0226, 0.0334, 0.0117, 0.0000, 0.0054, 0.0006, 0.0052, 0.0022, 0.0030, 0.0000, 0.0003, 0.0051, 0.0000, 0.0000, 0.0001, 0.0017, 0.0005, 0.0001, 0.0005, 0.0000, 0.0039, 0.0001, 0.0007} -> -2017.54996
2: {0.0370, 0.0137, 0.0371, 0.0273, 0.0363, 0.0139, 0.0000, 0.0047, 0.0010, 0.0032, 0.0017, 0.0039, 0.0002, 0.0007, 0.0060, 0.0000, 0.0001, 0.0008, 0.0021, 0.0006, 0.0001, 0.0004, 0.0009, 0.0103, 0.0001, 0.0004} -> -2027.02130
3: {0.0311, 0.0083, 0.0539, 0.0224, 0.0370, 0.0123, 0.0000, 0.0041, 0.0004, 0.0041, 0.0027, 0.0030, 0.0000, 0.0005, 0.0073, 0.0003, 0.0000, 0.0008, 0.0020, 0.0001, 0.0001, 0.0007, 0.0001, 0.0134, 0.0001, 0.0007} -> -2027.35409
4: {0.0320, 0.0061, 0.0438, 0.0277, 0.0363, 0.0165, 0.0005, 0.0058, 0.0006, 0.0044, 0.0019, 0.0038, 0.0004, 0.0009, 0.0074, 0.0002, 0.0000, 0.0004, 0.0030, 0.0003, 0.0002, 0.0017, 0.0032, 0.0090, 0.0002, 0.0011} -> -2025.32000
5: {0.0306, 0.0055, 0.0474, 0.0253, 0.0404, 0.0145, 0.0011, 0.0042, 0.0013, 0.0031, 0.0068, 

,i,lambda1,lambda2,lambda3,lambda4,lambda5,mu1,q12,q13,q14,⋯,q35,q41,q42,q43,q45,q51,q52,q53,q54,p
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0.03671910,0.017130258,0.04143393,0.02256943,0.03337229,0.011669090,2.565224e-06,0.005393185,0.0005922472,⋯,7.567896e-05,0.001693309,5.167964e-04,7.875869e-05,0.0004673921,2.180202e-05,0.003916093,1.486947e-04,0.0007347567,-2017.550
2,2,0.03702447,0.013736528,0.03707282,0.02733473,0.03629491,0.013891721,2.409875e-05,0.004659294,0.0010244302,⋯,7.846515e-04,0.002101102,6.458549e-04,6.918334e-05,0.0003539994,8.709720e-04,0.010306911,9.475016e-05,0.0003716734,-2027.021
3,3,0.03114631,0.008261170,0.05386707,0.02244770,0.03701997,0.012316779,3.892584e-05,0.004113752,0.0003735440,⋯,8.016044e-04,0.002008612,6.848271e-05,7.757612e-05,0.0007492349,5.941338e-05,0.013447248,1.356254e-04,0.0007017054,-2027.354
4,4,0.03201379,0.006094140,0.04380171,0.02772466,0.03632412,0.016502162,5.021894e-04,0.005788414,0.0006365430,⋯,3.563148e-04,0.002955069,3.212832e-04,2.491965e-04,0.0016704237,3.241098e-03,0.009019068,1.855770e-04,0.0010889214,-2025.320
5,5,0.03058287,0.005503584,0.04742589,0.02533857,0.04038314,0.014510785,1.073261e-03,0.004177817,0.0012632363,⋯,2.930451e-04,0.001185377,4.022389e-04,7.502182e-04,0.0017484151,3.050266e-03,0.007412734,4.591511e-04,0.0010279897,-2025.806
6,6,0.02669482,0.011130460,0.04118909,0.02701935,0.04038825,0.009348062,5.517540e-04,0.004796568,0.0011695855,⋯,7.493633e-05,0.002580160,1.335827e-03,5.918922e-04,0.0007367983,1.355973e-02,0.006495743,4.049816e-05,0.0003805312,-2017.482


### Burn-in period

In [114]:
#Burn-in period
mcmc_MuSSE_new <- subset(mcmc_MuSSE, i > 100)

pdf("Path/to/Appendix_burnin.pdf")

par(mfrow=c(1,1), mar=c(3,4,1,1))

plot(mcmc_MuSSE_new$i, mcmc_MuSSE_new$p, xlim=c(0,10000), ty="l", xlab="Generations", ylab="log-likelihood", bty="n", main="After a 10% burn-in", cex.main="0.8")

dev.off()

png 
  2

### Setting the diversification difference

In [99]:
# Setting the diversification difference
mcmc_MuSSEdiff <- with(mcmc_MuSSE_new, data.frame(S1 = lambda1, S2 = lambda2,  S3 = lambda3,   S4 = lambda4,   S5 = lambda5))

mcmc_MuSSEdiffS1 <- with(mcmc_MuSSE_new, data.frame(S1_2 = lambda1 - lambda2))
mcmc_MuSSEdiffS2 <- with(mcmc_MuSSE_new, data.frame(S1_3 = lambda1 - lambda3))
mcmc_MuSSEdiffS3 <- with(mcmc_MuSSE_new, data.frame(S2_3 = lambda2 - lambda3))
mcmc_MuSSEdiffS4 <- with(mcmc_MuSSE_new, data.frame(S1_4 = lambda1 - lambda4))
mcmc_MuSSEdiffS5 <- with(mcmc_MuSSE_new, data.frame(S1_5 = lambda1 - lambda5))
mcmc_MuSSEdiffS7 <- with(mcmc_MuSSE_new, data.frame(S2_4 = lambda2 - lambda4))
mcmc_MuSSEdiffS8 <- with(mcmc_MuSSE_new, data.frame(S1_5 = lambda2 - lambda5))
mcmc_MuSSEdiffS10 <- with(mcmc_MuSSE_new, data.frame(S3_4 = lambda3 - lambda4))
mcmc_MuSSEdiffS11 <- with(mcmc_MuSSE_new, data.frame(S3_5 = lambda3 - lambda5))
mcmc_MuSSEdiffS13 <- with(mcmc_MuSSE_new, data.frame(S4_5 = lambda4 - lambda5))

### Plot speciation 

In [96]:
# Plot speciation 

pdf("Path/to/Figure-MuSSE.pdf")

par( mar=c(3,4,1,1))

colors=c("deepskyblue","grey", "yellow", "red", "green")

profiles.plot(mcmc_MuSSEdiff[1:5], col.line=colors, xlim=c(0,0.07), xlab="", ylab="", las=1, bty="n", main="a) Speciation rates", cex.main=0.8, n.br = 100)

legend("topright", bty="n", c("Shelf shark", "Oceanic shark", "Small deep shark", "Small reef shark", "Big reef shark"), col= colors, lty=1,lwd="4",cex=0.6)

dev.off()

png 
  2

### Plot speciation difference

In [102]:
# Plot speciation difference
pdf("Path/to/Appendix-Differences-MuSSE-speciation-rates-MCMC.pdf")

par(mfrow=c(5,3), mar=c(3,4,1,1))

colors=c("deepskyblue","grey", "yellow", "red", "green")

#diff r1 and r2
profiles.plot(mcmc_MuSSEdiffS1[1], col.line=c("grey"), xlim=c(-0.05,0.05), xlab="", ylab="", las=1, bty="n", main="a) Difference between blue and grey diversification", cex.main=0.8)
abline(v=c(0),col="red")

#diff r1 and r3
profiles.plot(mcmc_MuSSEdiffS2[1], col.line=c("grey"), xlim=c(-0.15,0.05), xlab="", ylab="", las=1, bty="n", main="b) Difference between blue and yellow diversification", cex.main=0.8)
abline(v=c(0),col="red")

#diff r1 and r4
profiles.plot(mcmc_MuSSEdiffS4[1], col.line=c("grey"), xlim=c(-0.05,0.05), xlab="", ylab="", las=1, bty="n", main="d) Difference between blue and red diversification", cex.main=0.8)
abline(v=c(0),col="red")

#diff r1 and r5
profiles.plot(mcmc_MuSSEdiffS5[1], col.line=c("grey"), xlim=c(-0.15,0.05), xlab="", ylab="", las=1, bty="n", main="e) Difference between blue and green diversification", cex.main=0.8)
abline(v=c(0),col="red")

#diff r2 and r3
profiles.plot(mcmc_MuSSEdiffS3[1], col.line=c("grey"), xlim=c(-0.15,0.05), xlab="", ylab="", las=1, bty="n", main="c) Difference between grey and yellow diversification", cex.main=0.8)
abline(v=c(0),col="red")

#diff r2 and r4
profiles.plot(mcmc_MuSSEdiffS7[1], col.line=c("grey"), xlim=c(-0.05,0.05), xlab="", ylab="", las=1, bty="n", main="f) Difference between grey and red diversification", cex.main=0.8)
abline(v=c(0),col="red")

#diff r2 and r5
profiles.plot(mcmc_MuSSEdiffS8[1], col.line=c("grey"), xlim=c(-0.15,0.05), xlab="", ylab="", las=1, bty="n", main="g) Difference between grey and green diversification", cex.main=0.8)
abline(v=c(0),col="red")

#diff r3 and r4
profiles.plot(mcmc_MuSSEdiffS10[1], col.line=c("grey"), xlim=c(-0.05,0.05), xlab="", ylab="", las=1, bty="n", main="h) Difference between yellow and red diversification", cex.main=0.8)
abline(v=c(0),col="red")

#diff r3 and r5
profiles.plot(mcmc_MuSSEdiffS11[1], col.line=c("grey"), xlim=c(-0.15,0.05), xlab="", ylab="", las=1, bty="n", main="i) Difference between yellow and green diversification", cex.main=0.8)
abline(v=c(0),col="red")

#diff r4 and r5
profiles.plot(mcmc_MuSSEdiffS13[1], col.line=c("grey"), xlim=c(-0.05,0.05), xlab="", ylab="", las=1, bty="n", main="j) Difference between red and green diversification", cex.main=0.8)
abline(v=c(0),col="red")

dev.off()

png 
  2

## IV Ancestral state estimmation

In [ ]:
# IV Ancestral state estimmation

In [31]:
attach("Shark_MCMC_DATA")

L'objet suivant est masqué depuis file:Shark_MCMC_DATA (pos = 6):

    mcmc_MuSSE




### Applying the ancestral state estimmation for each trait

In [35]:
# Applying the ancestral state estimmation for each trait
st1<-apply(mcmc_MuSSE_new[2:27], 1, function(x) asr.marginal(lik.sp_qt, x)[1,])
st2<-apply(mcmc_MuSSE_new[2:27], 1, function(x) asr.marginal(lik.sp_qt, x)[2,])
st3<-apply(mcmc_MuSSE_new[2:27], 1, function(x) asr.marginal(lik.sp_qt, x)[3,])
st4<-apply(mcmc_MuSSE_new[2:27], 1, function(x) asr.marginal(lik.sp_qt, x)[4,])
st5<-apply(mcmc_MuSSE_new[2:27], 1, function(x) asr.marginal(lik.sp_qt, x)[5,])

In [38]:
st.m.avg1<-rowMeans(st1)
st.m.avg2<-rowMeans(st2)
st.m.avg3<-rowMeans(st3)
st.m.avg4<-rowMeans(st4)
st.m.avg5<-rowMeans(st5)

In [55]:
table_ASE<-as.data.frame(t(rbind(st.m.avg1, st.m.avg2, st.m.avg3, st.m.avg4, st.m.avg5)))

### Saving the probability table

In [351]:
# Saving the probability table
write.csv(as.data.frame(table_ASE), "Path/to/Table_MCMC_ASE.tsv", sep ="\t")

Warning message in write.csv(as.data.frame(xc), "Table_MCMC_ASE.tsv", sep = "\t"):
“attempt to set 'sep' ignored”


## V Plotting the Ancestral State Estimmation

In [ ]:
# V Plotting the Ancestral State Estimmation

### Generating the tree (semicircular)

In [46]:
# Generating the tree (semicircular)
phy_cil<-ggtree(nex3, layout="fan", open.angle=15)  + 
theme_bw() +
      theme(panel.border = element_blank(),
            legend.key = element_blank(),
           axis.ticks = element_blank(),
           axis.text.y = element_blank(),
           axis.text.x = element_blank(),
           panel.grid = element_blank(),
           panel.grid.minor = element_blank(), 
           panel.grid.major = element_blank(),
                   panel.background = element_blank(),
               plot.background = element_rect(fill = "transparent",colour = NA))

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.



### Generating pie chart for each node

In [58]:
# Generating pie chart for each node
table_ASE$node<-c(376:750) # selecting internal nodes
pies <- nodepie(table_ASE, cols=1:5, color=c('#9E0142', '#F46D43', '#FEE08B', '#ABDDA4', '#5E4FA2'), alpha=1)

Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”

Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”

Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”

Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”

Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”

Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”

Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”
Warning message in missingArg(color) || is.null(color) || is.na(color):
“‘length(x) = 5 > 1’ dans la conversion automatique vers ‘logical(1)’”

### Integrating pie chart into the phylogeny

In [59]:
# Integrating pie chart into the phylogeny
df_ASE<-tibble::tibble(node=as.numeric(table_ASE$node), pies=pies)
phy_cil2 <- phy %<+% df_ASE
ASE_plot<-phy_cil2 + geom_plot(data=td_filter(!isTip), mapping=aes(x=x,y=y, label=pies), vp.width=0.03, vp.height=0.03, hjust=0.5, vjust=0.5)

### Saving the tree

In [67]:
# Saving the tree
ggsave(ASE_plot, filename = "Path/to/output.pdf",  bg = "transparent", width = 10, height = 10)

Warning message:
“Removed 1 rows containing missing values (geom_plot).”
